In [149]:
import glob
import regex as re
import numpy as np
import pandas as pd
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
import geopy
from geopy.distance import vincenty
%matplotlib inline

In [21]:
path = '../data'

In [10]:
stations = pd.read_csv(path+'/stations.csv')
targets = pd.read_csv(path+'/targets.csv')
headers_mod = pd.read_csv(path+'/headers_mod.csv')
headers_obs = pd.read_csv(path+'/headers_obs.csv')

## Build raw data

In [144]:
with open('../data/headers_mod.csv') as f:
    mod_headers = f.readlines()[0].split()
    
with open('../data/headers_obs.csv') as f:
    obs_headers = f.readlines()[0].split()
    
mod_headers, obs_headers

(['lon', 'lat', 'day', 'hour', 'AirPollutant', 'Concentration'],
 ['Countrycode',
  'Namespace',
  'AirQualityNetwork',
  'AirQualityStation',
  'AirQualityStationEoICode',
  'SamplingPoint',
  'SamplingProcess',
  'Sample',
  'AirPollutant',
  'AirPollutantCode',
  'AveragingTime',
  'Concentration',
  'UnitOfMeasurement',
  'DatetimeBegin',
  'DatetimeEnd',
  'Validity',
  'Verification'])

In [ ]:
observations = []
for folder in glob.glob('../data/obs/*'):
    for file in glob.glob('{}/*'.format(folder)):
        year = re.search('\d{4}', file)[0]
        station = 'ES{}A'.format(re.findall('\d{4}', file)[1])
        data = pd.read_csv(file, sep='\t', names=obs_headers)
        observations.append((year, station, data))
        
observations[0][2].sample(5)

In [30]:
obs_df = pd.DataFrame()
for obs in observations:
    obs_df = obs_df.append(obs[2]).reset_index(drop=True)

In [33]:
obs_df.to_csv(path+'/all_obs.csv')

In [145]:
models = []
for folder in glob.glob('../data/mod/*'):
    for file in glob.glob('{}/*'.format(folder)):
        year = re.search('\d{4}', file)[0]
        station = 'ES{}A'.format(re.findall('\d{4}', file)[-1])
        data = pd.read_table(file, sep='\s+', names=mod_headers)
        data['year'] = year
        data['station'] = station
        models.append(data)
        
models = pd.concat(models)
models.sample(5)

,lon,lat,day,hour,AirPollutant,Concentration,year,station
36,2.14799,41.4261,2015-08-15,12:00:00,NO2,9.58821,2015,ES1856A
33,2.15339,41.3987,2013-11-22,09:00:00,NO2,18.27000,2013,ES1480A
27,2.14799,41.4261,2014-12-18,03:00:00,NO2,37.06750,2014,ES1856A
46,2.15339,41.3987,2013-02-13,22:00:00,NO2,34.73870,2013,ES1480A
22,2.15382,41.3853,2015-01-28,22:00:00,NO2,77.99080,2015,ES1438A


In [146]:
models = models.reset_index(drop=True)

In [147]:
models.to_csv(path+'/all_models.csv')

## Feature engineering

In [54]:
df_obs

,Countrycode,Namespace,AirQualityNetwork,AirQualityStation,AirQualityStationEoICode,SamplingPoint,SamplingProcess,Sample,AirPollutant,AirPollutantCode,AveragingTime,Concentration,UnitOfMeasurement,DatetimeBegin,DatetimeEnd,Validity,Verification,day,month,year
0,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,90,µg/m3,2013-07-05 18:00:00,2013-07-05 19:00:00,1,1,2013-07-05,7,2013
1,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,87,µg/m3,2013-07-05 19:00:00,2013-07-05 20:00:00,1,1,2013-07-05,7,2013
2,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,76,µg/m3,2013-07-05 20:00:00,2013-07-05 21:00:00,1,1,2013-07-05,7,2013
3,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,65,µg/m3,2013-07-05 21:00:00,2013-07-05 22:00:00,1,1,2013-07-05,7,2013
4,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,56,µg/m3,2013-07-05 22:00:00,2013-07-05 23:00:00,1,1,2013-07-05,7,2013
5,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,66,µg/m3,2013-07-05 23:00:00,2013-07-06 00:00:00,1,1,2013-07-05,7,2013
6,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,87,µg/m3,2013-07-06 00:00:00,2013-07-06 01:00:00,1,1,2013-07-06,7,2013
7,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,92,µg/m3,2013-07-06 01:00:00,2013-07-06 02:00:00,1,1,2013-07-06,7,2013
8,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,93,µg/m3,2013-07-06 02:00:00,2013-07-06 03:00:00,1,1,2013-07-06,7,2013
9,in/ES_8_2013-2015_timeseries.csv:ES,ES.BDCA.AQD,NET_ES209A,STA_ES1480A,ES1480A,SP_08019044_8_8,SPP_08019044_8_8.1,SAM_08019044_8_8,NO2,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,hour,85,µg/m3,2013-07-06 03:00:00,2013-07-06 04:00:00,1,1,2013-07-06,7,2013


In [126]:
df_obs['DatetimeBegin'] = pd.to_datetime(df_obs['DatetimeBegin'])
df_obs['DatetimeEnd'] = pd.to_datetime(df_obs['DatetimeEnd'])
df_obs['date'] = df_obs['DatetimeBegin'].dt.date
df_obs = df_obs.rename(columns={'AirQualityStationEoICode': 'station'})

In [127]:
daily_targets = df_obs.groupby(by=['date', 'station'])['Concentration'].max().reset_index()
daily_targets['target'] = 0
daily_targets.loc[daily_targets['Concentration'] > 100, 'target'] = 1

In [128]:
daily_targets[daily_targets['target'] == 1]

,date,station,Concentration,target
2,2013-01-01,ES1438A,112,1
9,2013-01-02,ES1438A,119,1
14,2013-01-03,ES0691A,119,1
16,2013-01-03,ES1438A,117,1
17,2013-01-03,ES1480A,102,1
20,2013-01-03,ES1992A,110,1
22,2013-01-04,ES1396A,113,1
23,2013-01-04,ES1438A,123,1
24,2013-01-04,ES1480A,187,1
25,2013-01-04,ES1679A,108,1


In [129]:
len(daily_targets[daily_targets['target'] == 1]) / len(daily_targets) * 100

19.49490474080638

In [130]:
len(daily_targets[daily_targets['target'] == 1])

1320

In [131]:
dias = [1, 29, 1, 1, 20, 24, 15, 11, 24, 12, 1, 6, 25, 26, 
        1, 6, 18, 21, 1, 9, 24, 15, 11, 24, 1, 6, 8, 25, 26,
        1, 6, 3, 6, 1, 1, 24, 15, 11, 24, 12, 8, 25, 26]
mes = [1, 3, 4, 5, 5, 6, 8, 9, 9, 10, 11, 12, 12, 12, 
       1, 1, 4, 4, 5, 6, 6, 8, 9, 9, 11, 12, 12, 12, 12,
       1, 1, 4, 4, 5, 6, 6, 8, 9, 9, 10, 12, 12, 12]
       
años = [2013] * 14 + [2014] * 15 + [2015] * 14

holidays = pd.DataFrame({'day': dias, 'month': mes, 'year': años})

In [132]:
def add_holidays(row):
    daily_targets.loc[(daily_targets['day'] == row['day']) & 
                      (daily_targets['month'] == row['month']) & 
                      (daily_targets['year'] == row['year']), 'holiday'] = 1

In [133]:
daily_targets['date'] = pd.to_datetime(daily_targets['date'])
daily_targets['year'] = daily_targets['date'].dt.year
daily_targets['month'] = daily_targets['date'].dt.month
daily_targets['week_day'] = daily_targets['date'].dt.dayofweek
daily_targets['day'] = daily_targets['date'].dt.day
daily_targets['weekend'] = 0
daily_targets.loc[daily_targets['week_day']>4, 'weekend'] = 1
daily_targets['holiday'] = 0
holidays.apply(add_holidays, axis = 1)
daily_targets.loc[daily_targets['week_day']>4, 'holiday'] = 1

In [134]:
daily_targets

,date,station,Concentration,target,year,month,week_day,day,weekend,holiday
0,2013-01-01,ES0691A,72,0,2013,1,1,1,0,1
1,2013-01-01,ES1396A,98,0,2013,1,1,1,0,1
2,2013-01-01,ES1438A,112,1,2013,1,1,1,0,1
3,2013-01-01,ES1480A,94,0,2013,1,1,1,0,1
4,2013-01-01,ES1679A,69,0,2013,1,1,1,0,1
5,2013-01-01,ES1856A,39,0,2013,1,1,1,0,1
6,2013-01-01,ES1992A,68,0,2013,1,1,1,0,1
7,2013-01-02,ES0691A,87,0,2013,1,2,2,0,0
8,2013-01-02,ES1396A,86,0,2013,1,2,2,0,0
9,2013-01-02,ES1438A,119,1,2013,1,2,2,0,0


In [120]:
daily_targets[(daily_targets['day'] == 1) & (daily_targets['month'] == 4) & (daily_targets['year'] == 2013)]

,date,AirQualityStation,Concentration,target,year,month,week_day,day,weekend,holiday
625,2013-04-01,STA_ES0691A,45,0,2013,4,0,1,0,0
626,2013-04-01,STA_ES1396A,31,0,2013,4,0,1,0,0
627,2013-04-01,STA_ES1438A,48,0,2013,4,0,1,0,0
628,2013-04-01,STA_ES1480A,50,0,2013,4,0,1,0,0
629,2013-04-01,STA_ES1679A,28,0,2013,4,0,1,0,0
630,2013-04-01,STA_ES1856A,33,0,2013,4,0,1,0,0
631,2013-04-01,STA_ES1992A,35,0,2013,4,0,1,0,0


In [124]:
daily_targets[daily_targets['holiday'] == 1]

,date,AirQualityStation,Concentration,target,year,month,week_day,day,weekend,holiday
0,2013-01-01,STA_ES0691A,72,0,2013,1,1,1,0,1
1,2013-01-01,STA_ES1396A,98,0,2013,1,1,1,0,1
2,2013-01-01,STA_ES1438A,112,1,2013,1,1,1,0,1
3,2013-01-01,STA_ES1480A,94,0,2013,1,1,1,0,1
4,2013-01-01,STA_ES1679A,69,0,2013,1,1,1,0,1
5,2013-01-01,STA_ES1856A,39,0,2013,1,1,1,0,1
6,2013-01-01,STA_ES1992A,68,0,2013,1,1,1,0,1
28,2013-01-05,STA_ES0691A,60,0,2013,1,5,5,1,1
29,2013-01-05,STA_ES1396A,114,1,2013,1,5,5,1,1
30,2013-01-05,STA_ES1480A,131,1,2013,1,5,5,1,1


In [140]:
for year in [2013, 2014, 2015]:
    daily_targets.loc[(daily_targets['date'] >= datetime.date(year-1, 12, 21)) & 
                      (daily_targets['date'] <= datetime.date(year, 3, 20)), 'season'] = 0
    daily_targets.loc[(daily_targets['date'] >= datetime.date(year, 3, 21)) &
                      (daily_targets['date'] <= datetime.date(year, 6, 20)), 'season'] = 1
    daily_targets.loc[(daily_targets['date'] >= datetime.date(year, 6, 21)) &
                      (daily_targets['date'] <= datetime.date(year, 9, 20)), 'season'] = 2
    daily_targets.loc[(daily_targets['date'] >= datetime.date(year, 9, 21)) &
                      (daily_targets['date'] <= datetime.date(year, 12, 20)), 'season'] = 3
    daily_targets.loc[(daily_targets['date'] >= datetime.date(year, 12, 21)) &
                      (daily_targets['date'] <= datetime.date(year+1, 3, 20)), 'season'] = 0

In [200]:
daily_targets.to_csv(path+'/daily_targets_dates.csv')

In [197]:
station = stations.rename(columns={'code': 'station'})

In [192]:
for st in stations['code']:
    stations['dist_to_'+st] = 0
    for i in stations.index:
        dist = vincenty((stations.loc[i,'lat'],stations.loc[i,'lon']), 
                        (stations[stations['code'] == st]['lat'].iloc[0], 
                         stations[stations['code'] == st]['lon'].iloc[0]))
        stations.loc[i, 'dist_to_'+st] = dist.km
        